In [6]:
import numpy as np
import tensorflow as tf
import pandas as pd
import librosa
import librosa.display
import os.path as p
%matplotlib inline

datasource_path = "UrbanSound8k"
audio_source = p.join(datasource_path, "audio")
metadata_path = p.join(p.join(datasource_path, "metadata"),"UrbanSound8k.csv")
metadata = pd.read_csv(metadata_path, index_col="fsID")
spectro_path = p.join("UrbanSound8k", "spectrogram")

folds = ["fold{}".format(str(i)) for i in range(1,11)]

In [9]:
import gc


def build_path(file_name,spectrogram = False):
    file= file_name.split(".")
    source = spectro_path if spectrogram else audio_source
    file_name = file[0] + ".jpg" if spectrogram else file[0] +".wav"
    fsID= int(file_name.split("-")[0])
    frame = metadata.loc[fsID]
    fold_id = 0
    try:
        fold_id = frame["fold"].values[0]-1
    except:
        fold_id = frame["fold"] -1
    file_with_fold = p.join(folds[fold_id], file_name)
    return p.join(source,file_with_fold), fold_id


def add_spectrogram_to_metadata():
    counter = 0
    errors = 0
    for index, row in metadata.iterrows():
        counter += 1
        if counter%2000 ==0:
            gc.collect()
        try:
            file_name = row["slice_file_name"]
            s,_ = build_path(file_name, True)
            metadata.loc[index, "spectro_path"] = s
            if not p.exists(s): create_spectrogram(row["slice_file_name"])
        except Exception as e:
            errors +=1

    print("Total: {} \nErrors: {}\nCorrect: {}".format(counter, errors, counter-errors))
    return metadata

def plot_spectrogram(filename):
    plt.interactive(False)
    filename, _ = build_path(filename)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig, ax = plt.subplots()
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    plt.show()

    
def create_spectrogram(filename):
    plt.interactive(False)
    filename, fold_id =build_path(filename)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    curr_path = p.join(spectro_path, folds[fold_id])
    savefile  = p.join(curr_path, "{}.jpg".format(p.basename(filename).split(".")[0] ))
    plt.savefig(savefile, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,clip,sample_rate,fig,ax,S,savefile

In [ ]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

In [10]:
add_spectrogram_to_metadata()
metadata[:5]

Total: 8732 
Errors: 107
Correct: 8625


,slice_file_name,start,end,salience,fold,classID,class,spectro_path
fsID,,,,,,,,
100032,100032-3-0-0.wav,0.0,0.317551,1,5,3,dog_bark,UrbanSound8k\spectrogram\fold5\100032-3-0-0.jpg
100263,100263-2-0-117.wav,58.5,62.500000,1,5,2,children_playing,UrbanSound8k\spectrogram\fold5\100263-2-0-36.jpg
100263,100263-2-0-121.wav,60.5,64.500000,1,5,2,children_playing,UrbanSound8k\spectrogram\fold5\100263-2-0-36.jpg
100263,100263-2-0-126.wav,63.0,67.000000,1,5,2,children_playing,UrbanSound8k\spectrogram\fold5\100263-2-0-36.jpg
100263,100263-2-0-137.wav,68.5,72.500000,1,5,2,children_playing,UrbanSound8k\spectrogram\fold5\100263-2-0-36.jpg


In [8]:
from PIL import Image

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

world
